In [1]:
import pandas as pd
from sqlalchemy import create_engine
import geopandas as gpd

import os

# Database connection details from zshrc environment variables
db_name = 'colorado_spills'
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')


# Create an engine to connect to the PostgreSQL database
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db_name}')

# Read the spills_with_demographics data from the database
df = pd.read_sql_table('spills_with_demographics', engine)





In [2]:
# Display the first few rows of the Spill Description column
df['Spill Description'].head()


0    Crews working on 6/11/14 in area of former pro...
1    Historical release discovered during removal o...
2    Historical release discovered during removal o...
3    The night operator noticed a high level alarm ...
4    On May 24, 2014, in anticipation of potential ...
Name: Spill Description, dtype: object

In [3]:
import re
import pandas as pd

# Text preprocessing function
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text

# Apply the preprocessing to the Spill Description column
df['Cleaned_Description'] = df['Spill Description'].apply(preprocess_text)

# Display the cleaned text for the first few rows
df['Cleaned_Description'].head()


0    crews working on 61114 in area of former produ...
1    historical release discovered during removal o...
2    historical release discovered during removal o...
3    the night operator noticed a high level alarm ...
4    on may 24 2014 in anticipation of potential fl...
Name: Cleaned_Description, dtype: object

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=2, stop_words='english')

# Fit and transform the cleaned descriptions
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Cleaned_Description'])

# Get the feature names (i.e., the words)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame with TF-IDF scores
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Display the top 10 words with the highest average TF-IDF score
top_keywords = tfidf_df.mean().sort_values(ascending=False).head(10)
top_keywords


water         0.051375
release       0.047366
soil          0.038160
location      0.034915
tank          0.032697
produced      0.032328
discovered    0.031280
line          0.029943
activities    0.028456
impacted      0.026087
dtype: float64

In [6]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 83.6 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
import spacy

# Load the pre-trained NER model from spacy
nlp = spacy.load("en_core_web_sm")

# Function to extract named entities
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Apply the NER extraction to the cleaned descriptions
df['Entities'] = df['Cleaned_Description'].apply(extract_entities)

# Display the entities for the first few rows
df[['Cleaned_Description', 'Entities']].head()


,Cleaned_Description,Entities
0,crews working on 61114 in area of former produ...,"[(61114, CARDINAL), (61114, CARDINAL)]"
1,historical release discovered during removal o...,[]
2,historical release discovered during removal o...,[]
3,the night operator noticed a high level alarm ...,[]
4,on may 24 2014 in anticipation of potential fl...,"[(may 24 2014, DATE), (m365636736, PERSON), (8..."


In [8]:
print(df.columns.tolist())


['Document #', 'Report', 'Operator', 'Operator #', 'Tracking #', 'Initial Report Date', 'Date of Discovery', 'Spill Type', 'Qtr Qtr', 'Section', 'Township', 'range', 'meridian', 'Latitude', 'Longitude', 'Municipality', 'county', 'Facility Type', 'Facility ID', 'API County Code', 'API Sequence Number', 'Spilled outside of berms', 'More than five barrels spilled', 'Oil Spill Volume', 'Condensate Spill Volume', 'Flow Back Spill Volume', 'Produced Water Spill Volume', 'E&P Waste Spill Volume', 'Other Waste', 'Drilling Fluid Spill Volume', 'Current Land Use', 'Other Land Use', 'Weather Conditions', 'Surface Owner', 'Surface Owner Other', 'Waters of the State', 'Residence / Occupied Structure', 'livestock', 'Public Byway', 'Surface Water Supply Area', 'Spill Description', 'Supplemental Report Date', 'Oil BBLs Spilled', 'Oil BBLs Recovered', 'Oil Unknown', 'Condensate BBLs Spilled', 'Condensate BBLs Recovered', 'Condensate Unknown', 'Produced Water BBLs Spilled', 'Produced Water BBLs Recovere